In [321]:

import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [322]:
df_train = pd.read_csv("train_data.csv")
df_test = pd.read_csv("test_data.csv")

In [323]:
df_train.describe()

,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Cabin,Embarked,Age
count,1493.000000,1493.000000,1493.000000,1493.000000,1493.000000,1493.000000,1493.000000,1493.000000,1.488000e+03
mean,597.697254,0.327528,2.261219,0.690556,0.184863,0.178835,5.994642,1.834561,2.151174e+05
std,251.365652,0.469469,0.628908,0.462419,0.594308,0.592203,24.214966,0.538178,8.295612e+06
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-3.200000e+04
25%,377.000000,0.000000,2.000000,0.000000,0.000000,0.000000,-1.000000,2.000000,-3.500000e+01
50%,758.000000,0.000000,2.000000,1.000000,0.000000,0.000000,-1.000000,2.000000,1.600000e+01
75%,813.000000,1.000000,3.000000,1.000000,0.000000,0.000000,-1.000000,2.000000,2.400000e+01
max,891.000000,1.000000,3.000000,1.000000,5.000000,6.000000,133.000000,2.000000,3.200000e+08


In [324]:
(df_train["Age"] < 0).sum()

679

In [325]:
df_train.drop(df_train[df_train.Age < 0].index, inplace = True)

In [326]:
(df_train["Age"] < 0).sum()

0

In [327]:
ageNa = df_test["Age"].isna().sum()

print(f"Ammount of Nan Values in Age Series: {ageNa}")

Ammount of Nan Values in Age Series: 0


In [328]:
(df_train["Age"] > 100).sum()

6

In [329]:
df_train.drop(df_train[df_train.Age > 100].index, inplace = True)

In [330]:
df_train["PassengerId"] = df_train["PassengerId"].drop_duplicates()

In [331]:
def sjekk_df(x):
    try:
        return float(x)
    except:
        return -1 

In [332]:

df_train["Fare"] = df_train.Fare.apply(lambda x: sjekk_df(x))

In [333]:
df_train["Ticket"] = df_train["Ticket"].drop_duplicates()

In [334]:
df_train = df_train.dropna()

In [335]:
df_train["Ticket"] = df_train.Fare.apply(lambda x: sjekk_df(x))

In [336]:
df_train.info()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 436 entries, 0 to 1492
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  436 non-null    float64
 1   Survived     436 non-null    int64  
 2   Pclass       436 non-null    int64  
 3   Sex          436 non-null    int64  
 4   SibSp        436 non-null    int64  
 5   Parch        436 non-null    int64  
 6   Ticket       436 non-null    float64
 7   Fare         436 non-null    float64
 8   Cabin        436 non-null    int64  
 9   Embarked     436 non-null    int64  
 10  Age          436 non-null    float64
dtypes: float64(4), int64(7)
memory usage: 40.9 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  143 non-null    int64  
 1   Survived     143 non-null    int64  
 2   Pclass       143 no

In [337]:
model = xgb.XGBRegressor()

In [338]:
X_train = df_train.drop("Age", axis=1)
y_train = df_train["Age"]

In [339]:
X_train, X_train_test, y_train, y_train_Test = train_test_split(X_train, y_train, train_size=0.8, random_state=42)

In [340]:
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [341]:
preds = model.predict(X_train_test)

In [342]:
mse = mean_squared_error(preds, y_train_Test)
mse_root = np.sqrt(mse)

In [343]:
print(mse_root)

14.351054823483793


In [344]:
params={
    "learning_rate":[0.05, 0.10, 0.15,0.20, 0.25, 0.30],
    "max_depth":[3,4,5,6,8,10,12,15],
    "min_child_weight":[1,3,5,7],
    "gamma":[0.0,0.1,0.2,0.3,0.4],
    "colsample_bytree":[0.3,0.4,0.5,0.7],
    "n_estimators":[100,200,300,400,500,900,1100,1500]

}

In [345]:
model2 = xgb.XGBRegressor()

random_search = RandomizedSearchCV(model, param_distributions=params, n_iter=250, scoring="neg_mean_squared_error", n_jobs=-1, cv=5)

# Fit the model with x and y train sets
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          callbacks=None, colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=0, gpu_id=-1,
                                          grow_policy='depthwise',
                                          importance_type=None,
                                          interaction_constraints='',
                                          learning_rate=0.3000...
                                          num_parallel_tree=1, predictor='auto',
                                          random_state=0, ...),
                   n_iter=250, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.3, 0.4, 0.5,
                                                             0.7],
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                                        'learning_rate': [0.05, 0.1, 0.15, 0.2,
                                                          0.25, 0.3],
                                        'max_depth': [3, 4, 5, 6, 8, 10, 12,
                                                      15],
                                        'min_child_weight': [1, 3, 5, 7],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 900, 1100,
                                                         1500]},
                   scoring='neg_mean_squared_error')

In [346]:
model_new = random_search.best_params_
model_new

{'n_estimators': 100,
 'min_child_weight': 3,
 'max_depth': 3,
 'learning_rate': 0.1,
 'gamma': 0.0,
 'colsample_bytree': 0.5}

In [347]:
model_new = random_search.best_estimator_

In [348]:
preds = model_new.predict(X_train_test)

In [349]:
mse_new = mean_squared_error(preds, y_train_Test)
mse_root_new = np.sqrt(mse_new)
print(mse_root_new)

13.638754655285892


In [350]:
X_test = df_test.drop("Age", axis=1)
Y_test = df_test["Age"]

In [351]:

preds_test = model_new.predict(X_test)

In [355]:
Y_test.info()

<class 'pandas.core.series.Series'>
RangeIndex: 143 entries, 0 to 142
Series name: Age
Non-Null Count  Dtype  
--------------  -----  
143 non-null    float64
dtypes: float64(1)
memory usage: 1.2 KB


In [356]:
mse_test=mean_squared_error(preds_test,Y_test)

In [357]:

mse_test_root = np.sqrt(mse_test)
print(mse_test_root)


11.680836766631199


In [359]:
print(f'relation between better error on the new model and the old error: {mse_test_root/mse_root_new}')

relation between better error on the new model and the old error: 0.8564445260479941
